# Case - Machine Learning Engineer - PicPay

Este material tem por objetivo trazer uma análise dos dados disponibilizados para o case técnico relativo ao processo de preenchimento de vaga de machine learning engineer. Neste trabalho, serão disponibilizados os processamentos necessários para a resposta das perguntas fornecidas, bem como a análise de métricas para a criação do modelo utilizado na API.

## Importando Base de Dados

In [42]:
from pyspark.sql.functions import (
    isnan, col, desc, mean, count, row_number, min, max, concat_ws, to_date, date_format
)
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
import zipfile
import os

In [2]:
filepath = os.path.join(os.getcwd(), "airports-database.zip")

with zipfile.ZipFile(filepath, 'r') as zip_file:
    zip_file.extractall()

In [3]:
spark=SparkSession.builder.config(
    "spark.driver.bindAddress", "localhost"
    ).config(
        "spark.network.timeout","10000000"
    ).config(
        "spark.executor.heartbeatInterval", "10000000"
    ).appName("MLE Case").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/21 21:19:54 WARN Utils: Your hostname, Notebook, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/21 21:19:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/21 21:19:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
csv_filepath = os.path.join(os.getcwd(), "airports-database.csv")

df = spark.read.csv(csv_filepath, header=True, inferSchema=True)

df.show()

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
|  0|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|United Air Lines ...|
|  1|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00|United Air Lines ...|
|  2|2013|    1|  1|   54

In [5]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: double (nullable = true)
 |-- sched_dep_time: integer (nullable = true)
 |-- dep_delay: double (nullable = true)
 |-- arr_time: double (nullable = true)
 |-- sched_arr_time: integer (nullable = true)
 |-- arr_delay: double (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: double (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- time_hour: timestamp (nullable = true)
 |-- name: string (nullable = true)



In [6]:
df.count()

336776

## Respostas das perguntas

#### 1. Qual o número total de voos no conjunto de dados?

In [7]:
distinct_flights = df.select('flight').distinct()

distinct_flights.dropDuplicates().count()

3844

Portanto, o número total é de 3844 voos.

#### 2. Quantos voos foram cancelados?

In [8]:

df.filter(
    (isnan(col('dep_time')) | col('dep_time').isNull()) &
    (isnan(col('arr_time')) | col('arr_time').isNull())
).count()

8255

O número de voos cancelados, considerando a situação onde tanto a coluna `arr_time` quanto `dep_time` são nulos é de 8255 vôos.

#### 3. Quais são os 5 aeroportos com maior número de pousos?

In [9]:
res = df.groupBy('dest').count().orderBy(desc('count'))

print("5 aeroportos com maior número de pousos:")
res.show(5)

5 aeroportos com maior número de pousos:


+----+-----+
|dest|count|
+----+-----+
| ORD|17283|
| ATL|17215|
| LAX|16174|
| BOS|15508|
| MCO|14082|
+----+-----+
only showing top 5 rows


Assim, temos que os aeroportos com maior número de pousos são: O'Hare International Airport, em Chicago, EUA; Hartsfield-Jackson Atlanta International Airport, em Atlanta, EUA; Los Angeles International Airport, em Los Angeles, EUA; Logan International Airport, em Boston, EUA; Orlando International Airport, em Orlando, EUA.

#### 4. Qual a rota mais frequente (par origin-dest)?

In [10]:
df.groupBy(['origin', 'dest']).count().orderBy(desc('count')).show(1)

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   JFK| LAX|11262|
+------+----+-----+
only showing top 1 row


Portanto, temos que a rota mais frequênte parte do John F. Kennedy International Airport, em Nova Iorque, EUA, com destino ao Los Angeles International Airport, Los Angeles, EUA.

#### 5. Quais são as 5 companhias aéreas com maior tempo médio de atraso na chegada?

In [11]:
df.groupBy('carrier')\
    .agg(mean('arr_delay').alias('mean_delay'))\
    .orderBy(col('mean_delay').desc()).show(5)


+-------+------------------+
|carrier|        mean_delay|
+-------+------------------+
|     F9|21.920704845814978|
|     FL|20.115905511811025|
|     EV| 15.79643108710965|
|     YV|15.556985294117647|
|     OO|11.931034482758621|
+-------+------------------+
only showing top 5 rows


Temos evidências de que as 5 companhias aéreas com maior tempo médio de atraso na chegada são:
1. Frontier Airlines;
2. Fly Play;
3. EVA Air;
4. Mesa Airlines;
5. SkyWest Airlines.

#### 6. Qual é o dia da semana com maior número de vôos?

In [53]:
date_df = df.select('year', 'month', 'day').withColumn('date_as_str', concat_ws('-', 'year', 'month', 'day'))
date_df = date_df.withColumn('date', to_date('date_as_str', 'yyyy-M-d'))
date_df = date_df.withColumn('weekday', date_format('date', 'EEEE'))
date_df = date_df.groupBy('weekday').agg(count('*').alias('count_weekday')).orderBy(desc('count_weekday'))
date_df.show()

+---------+-------------+
|  weekday|count_weekday|
+---------+-------------+
|   Monday|        50690|
|  Tuesday|        50422|
|   Friday|        50308|
| Thursday|        50219|
|Wednesday|        50060|
|   Sunday|        46357|
| Saturday|        38720|
+---------+-------------+



Portanto, o dia da semana com maior número de voos é a Segunda-Feira.

#### 7. Qual a rota que teve maior tempo de vôo médio (air_time)?

In [12]:
df.groupBy(['origin', 'dest']).agg(mean('air_time').alias('mean_air_time')).orderBy(desc('mean_air_time')).show(1)

+------+----+-----------------+
|origin|dest|    mean_air_time|
+------+----+-----------------+
|   JFK| HNL|623.0877192982456|
+------+----+-----------------+
only showing top 1 row


A rota com maior tempo de vôo médio é a que parte do John F. Kennedy International Airport, em Nova Iorque, EUA, com destino ao Honolulu International Airport, Honolulu, EUA

#### 8. Para cada aeroporto de origem, qual é o aeroporto de destino mais comum?

In [ ]:
count_dest = df.groupBy('origin', 'dest').agg(count('*').alias('count_dest'))
window_func = Window.partitionBy('origin').orderBy(col('count_dest').desc())

wind_count_dest = count_dest.withColumn('pos', row_number().over(window_func))

most_common_dest = wind_count_dest.filter(col('pos') == 1).drop('pos')

most_common_dest.show()

+------+----+----------+
|origin|dest|count_dest|
+------+----+----------+
|   EWR| ORD|      6100|
|   JFK| LAX|     11262|
|   LGA| ATL|     10263|
+------+----+----------+



Daí, é possível verificar que o destino mais comum é:
* O'Hare International Airport, Chicaco, EUA quando saindo de Newark Liberty Airport, Newark, EUA;
* John F. Kennedy International Airport, Nova Iorque, EUA quando saindo de Los Angeles International Airport, Los Angeles, EUA;
* Atlantic City International Airport, Atlantic City, EUA quando saindo de LaGuardia Airport, Nova Iorque, EUA.

#### 9. Quais são as 3 rotas que tiveram maior variação no tempo médio de voo (air_time)?

In [33]:
df.groupBy('origin', 'dest')\
    .agg(
        min('air_time').alias('min_time'),
        max('air_time').alias('max_time')
    ).withColumn('variation', col('max_time') - col('min_time'))\
    .orderBy(col('variation').desc()).show(3)

+------+----+--------+--------+---------+
|origin|dest|min_time|max_time|variation|
+------+----+--------+--------+---------+
|   JFK| SFO|   301.0|   490.0|    189.0|
|   JFK| LAX|   275.0|   440.0|    165.0|
|   JFK| EGE|   219.0|   382.0|    163.0|
+------+----+--------+--------+---------+
only showing top 3 rows


Assim, temos as rotas com maior variação no tempo de vôo sendo:
* John F. Kennedy International Airport até San Francisco International Airport;
* John F. Kennedy International Airport até Los Angeles International Airport;
* John F. Kennedy International Airport até Eagle County Regional Airport.

#### 10. Quais são as 3 rotas mais comuns que tiveram atrasos na chegada superiores a 30 minutos?

In [40]:
df.groupby('origin', 'dest').agg(count(col('arr_delay') > 30).alias('delay')).orderBy(desc('delay')).show(3)

+------+----+-----+
|origin|dest|delay|
+------+----+-----+
|   JFK| LAX|11159|
|   LGA| ATL|10041|
|   LGA| ORD| 8507|
+------+----+-----+
only showing top 3 rows


Portanto, as rotas são:
* John F. Kennedy International Airport até Los Angeles International Airport;
* LaGuardia Airport até Hartsfield-Jackson Atlanta International Airport;
* LaGuardia Airport até O'Hare International Airport